# Video Processing for BT Lab
This notebook to test risky code fragments
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [64]:
#import libraries
import os
import cv2
import json
import re
import datetime
import time
import numpy as np
import pandas as pd

In [ ]:
def get_video_duration(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)

    # Get the frames per second (fps) of the video
    fps = video_capture.get(cv2.CAP_PROP_FPS)

    # Calculate the duration in seconds
    duration_seconds = round(total_frames / fps)

    # Convert the duration to a human-readable format (HH:MM:SS)
    video_time = datetime.timedelta(seconds=duration_seconds)

    # Close the video capture object
    video_capture.release()

    return f"duration in seconds: {duration_seconds}, video time: {video_time}"


In [ ]:
def count_frames(video_path):
    video = cv2.VideoCapture(video_path)
    success, frame = video.read()
    frame_count = 0

    while success:
        success, frame = video.read()
        
        if not success:
            break

        timestamp_ms = video.get(cv2.CAP_PROP_POS_MSEC)
        timestamp_sec = timestamp_ms / 1000.0
        print("time: ", timestamp_sec, "frame: ", frame_count)
        print(success, frame)
        frame_count += 1

    print("Manual Frame Count: ", frame_count)
    video.release()

In [56]:
def re_sample(old_fps: int, new_fps: int) -> list[int]:
    frames_arr = np.arange(0, old_fps, dtype=int)
    frames_interp = np.linspace(0, old_fps - 1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()